In [1]:
import sys
sys.path.append('../lib/')

import pandas as pd
import numpy as np
import geopandas as gpd

import workers
import v_ij
import math
import sweden

from shapely.geometry import Point
import gc
from copy import deepcopy

### load 2km * 2km zone

In [2]:
grids = gpd.read_file('../results/grids_vgr_2km_density_deso5.shp')


In [3]:
grids.head()

,density,zone,deso_5,geometry
0,26.055,1,1401A,"POLYGON ((342000.000 6390000.000, 342000.000 6..."
1,26.055,2,1401A,"POLYGON ((342000.000 6396000.000, 342000.000 6..."
2,26.055,3,1401A,"POLYGON ((344000.000 6390000.000, 344000.000 6..."
3,26.055,4,1401A,"POLYGON ((344000.000 6392000.000, 344000.000 6..."
4,26.055,5,1401A,"POLYGON ((344000.000 6394000.000, 344000.000 6..."


### Get distance between zones

In [4]:
# This gives a stacked version
distances = workers.zone_distances(grids)
# This gives a matrix-style dataframe
df_d = distances.unstack(level=1)

Calculating distances between zones...


### Get area and average radius

In [5]:
area = 4 # km

# A=pi*r_average^2

r_average = math.sqrt(area/math.pi) # km

### Get population density and geometry

In [6]:
grid_name = list(grids.zone)
population_density = dict(zip(grids.zone, grids.density))
geometry = dict(zip(grids.zone, grids.geometry))

### Get $v^{tot}_{ij}$

In [7]:
# parameter = ln(f_max/f_min), f_min = 1/T, f_max = 1 T = 1000
T = 1000
f_max = 1
f_min = 1/T
parameter = math.log(f_max / f_min)

#ODM = {orig: {desti: v_{ori, desti}}}
# orig is the grid_name
# desti is the grid_name 


ODM_tot = dict()
for i in range(0, len(grid_name)):
    element = dict()
    for j in range(i + 1, len(grid_name)):
        number_of_trips = v_ij.average_daily_trips(population_density[grid_name[j]], area, r_average, df_d[grid_name[i]][grid_name[j]], parameter) + v_ij.average_daily_trips(population_density[grid_name[i]], area, r_average, df_d[grid_name[i]][grid_name[j]], parameter)
        element[grid_name[j]] = number_of_trips
    ODM_tot[grid_name[i]] = element

### Save ODM_tot between grids

In [8]:
with open('../results/ODM_grid.txt', 'w') as f:
   for k in ODM_tot.keys():
       for i in ODM_tot[k].keys():
           f.write(str(k))
           f.write('\t')
           f.write(str(i)) 
           f.write('\t')
           f.write(str(ODM_tot[k][i]))
           f.write('\n')  

### Load the ODM_tot between grids

In [9]:
ODM_tot = dict()
with open('../results/ODM_grid.txt', 'r') as f:
    line = f.readline()
    inner = dict()
    while line:
        data = line.split()
        original = int(data[0])
        destination = int(data[1])
        number_of_trip = float(data[2])
        inner[destination] = number_of_trip

        if destination == grid_name[-1]:
            ODM_tot[original] = deepcopy(inner)
            inner.clear()
        line = f.readline()

### Load Sweden VG zone data

In [10]:
zones = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
zones.loc[:, 'deso_3'] = zones.loc[:, 'deso'].apply(lambda x: x[:2])
zones_subset = zones.loc[zones['deso_3'] == '14', :]
zones_subset_info = dict(zip(zones_subset['deso'], zones_subset['geometry']))
zone_name = list(zones_subset['deso'])

### Aggregate level

In [11]:
zones_subset.loc[:, 'deso_5'] = zones_subset.loc[:, 'deso'].apply(lambda x: x[:5])

D:\Anaconda\envs\geopandas_lab\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
zones_subset.groupby(['deso_5'])

In [13]:
zones_subset.head()

,id,objectid,uuid,deso,befolkning,version,geometry,deso_3,deso_5
3563,3564.0,3564,{6349809D-EFF4-4CF3-8B17-FA4B18460A06},1401A0010,2278,2018_v2,"POLYGON ((344860.657 6401921.547, 345105.014 6...",14,1401A
3564,3565.0,3565,{972D1560-D815-4CBD-90C8-10E54B1AB758},1401A0020,1584,2018_v2,"POLYGON ((343371.029 6401284.527, 343383.043 6...",14,1401A
3565,3566.0,3566,{AC463249-527F-42FF-AC3C-217C812B13F3},1401B2010,1604,2018_v2,"POLYGON ((350267.572 6400639.429, 350153.500 6...",14,1401B
3566,3567.0,3567,{82429698-A197-4A31-BFA6-2F2C6C4BB0B7},1401B2020,1402,2018_v2,"POLYGON ((347372.465 6401230.998, 347671.575 6...",14,1401B
3567,3568.0,3568,{B6D36006-B7AF-49C8-918B-F8D923543BEC},1401B3010,1702,2018_v2,"POLYGON ((350830.549 6394368.985, 350866.237 6...",14,1401B


### Get zone level $V_{ij}$

In [14]:
cover = []
checked = set()
for i in range(0, len(zone_name)):
    sub_cover = []
    for j in range(0, len(grid_name)):
        if j in checked:
            continue
        point_j = Point(geometry[grid_name[j]].centroid.x, geometry[grid_name[j]].centroid.y)
        if zones_subset_info[zone_name[i]].contains(point_j) == True:
            # grid_j in zone_i
            sub_cover.append(grid_name[j])
            checked.add(j)   # This grid has been occupied, we do not need to check it again.
    cover.append(sub_cover)
within = dict(zip(zone_name, cover))

In [15]:
bigzone_name = []
bigCover = []
subCover = []
old_name = zone_name[0][0:5]

bigzone_name.append(old_name)
subCover.extend(within[zone_name[0]])

for i in range(1, len(zone_name)):
    new_name = zone_name[i][0:5]

    if new_name == old_name:
        # this two zones belong the same big zone
        subCover.extend(within[zone_name[i]])

    if new_name != old_name:
        # find a new big zone
        #store old results
        bigCover.append(deepcopy(subCover))
        subCover.clear()

        #store new resutls
        bigzone_name.append(new_name)
        subCover.extend(within[zone_name[i]])

    old_name = new_name

# handle the lastest case
bigCover.append(subCover)


big_within = dict(zip(bigzone_name, bigCover))

In [16]:
ODM_big = dict()
for i in range(0, len(bigzone_name)):
    element = dict()
    for j in range(i, len(bigzone_name)):
        if i == j:
            average_daily_trips = 0
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(begin + 1, len(big_within[bigzone_name[i]])):
                    average_daily_trips = average_daily_trips + ODM_tot[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[i]][end]]

            element[bigzone_name[j]] = 2 * average_daily_trips
        if i != j:
            average_daily_trips = 0
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[j]])):
                    average_daily_trips = average_daily_trips + ODM_tot[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]]
            element[bigzone_name[j]] = average_daily_trips

    ODM_big[bigzone_name[i]] = element

with open('../results/ODM_big.txt', 'w') as f:
   for k in ODM_big.keys():
       for i in ODM_big[k].keys():
           f.write(str(k))
           f.write('\t')
           f.write(str(i)) 
           f.write('\t')
           f.write(str(ODM_big[k][i]))
           f.write('\n')  